# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("project").getOrCreate()

In [3]:
df = spark.read.csv('cruise_ship_info.csv', inferSchema=True, header=True)

In [4]:
df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [5]:
from pyspark.ml.feature import StringIndexer

In [6]:
#index string value column
indexer = StringIndexer(inputCol="Cruise_line", outputCol='Cruise_line_index')
df_index = indexer.fit(df).transform(df)

In [7]:
df_index.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_index|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|             16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|             16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|              1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|              1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|              1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|       

In [8]:
df.select("Cruise_line").distinct().show()

+-----------------+
|      Cruise_line|
+-----------------+
|            Costa|
|              P&O|
|           Cunard|
|Regent_Seven_Seas|
|              MSC|
|         Carnival|
|          Crystal|
|           Orient|
|         Princess|
|        Silversea|
|         Seabourn|
| Holland_American|
|         Windstar|
|           Disney|
|        Norwegian|
|          Oceania|
|          Azamara|
|        Celebrity|
|             Star|
|  Royal_Caribbean|
+-----------------+



In [14]:
df_index.select("Cruise_line_index").distinct().show()

+-----------------+
|Cruise_line_index|
+-----------------+
|              8.0|
|              0.0|
|              7.0|
|             18.0|
|              1.0|
|              4.0|
|             11.0|
|             14.0|
|              3.0|
|             19.0|
|              2.0|
|             17.0|
|             10.0|
|             13.0|
|              6.0|
|              5.0|
|             15.0|
|              9.0|
|             16.0|
|             12.0|
+-----------------+



In [15]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [16]:
df_index.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_index']

In [17]:
assembler = VectorAssembler(inputCols=['Age', 'Tonnage', 'passengers', 'length','cabins', 
                                       'passenger_density','Cruise_line_index'], outputCol='features')

In [18]:
output = assembler.transform(df_index)

In [19]:
output.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- Cruise_line_index: double (nullable = true)
 |-- features: vector (nullable = true)



In [20]:
final_data = output.select('features', 'crew')

In [21]:
final_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [22]:
training_data, testing_data = final_data.randomSplit([0.7, 0.3])

In [23]:
from pyspark.ml.regression import LinearRegression

In [24]:
lr = LinearRegression(labelCol='crew')

In [25]:
lr_model = lr.fit(training_data)

In [30]:
lr_model.summary.pValues

[0.758941653839043,
 0.9274905405774276,
 0.012805710871613085,
 0.0058621352417873585,
 4.4583854164059744e-08,
 0.8214757871143081,
 0.13894935428862176,
 0.22987318391084477]

In [37]:
lr_model.summary.coefficientStandardErrors

[0.017529643127496615,
 0.016190613681732573,
 0.0629829180806249,
 0.15249772253614713,
 0.16149190779699824,
 0.021033746879969523,
 0.025201478010293814,
 1.3065451474892902]

In [31]:
lr_model.summary.featuresCol

'features'

In [32]:
test_results = lr_model.evaluate(testing_data)

In [34]:
testing_data.head(10)

[Row(features=DenseVector([4.0, 220.0, 54.0, 11.82, 27.0, 40.74, 0.0]), crew=21.0),
 Row(features=DenseVector([5.0, 115.0, 35.74, 9.0, 15.32, 32.18, 9.0]), crew=12.2),
 Row(features=DenseVector([5.0, 122.0, 28.5, 10.33, 6.87, 34.57, 6.0]), crew=6.7),
 Row(features=DenseVector([6.0, 113.0, 37.82, 9.51, 15.57, 29.88, 2.0]), crew=12.0),
 Row(features=DenseVector([7.0, 89.6, 25.5, 9.61, 12.75, 35.14, 7.0]), crew=9.87),
 Row(features=DenseVector([7.0, 116.0, 31.0, 9.51, 15.57, 37.42, 2.0]), crew=12.0),
 Row(features=DenseVector([8.0, 77.499, 19.5, 8.56, 9.75, 39.74, 2.0]), crew=9.0),
 Row(features=DenseVector([9.0, 81.0, 21.44, 9.21, 10.72, 37.78, 4.0]), crew=10.0),
 Row(features=DenseVector([9.0, 105.0, 27.2, 8.9, 13.56, 38.6, 5.0]), crew=10.68),
 Row(features=DenseVector([10.0, 46.0, 7.0, 6.7, 1.82, 65.71, 10.0]), crew=4.47)]

In [35]:
test_results.rootMeanSquaredError

0.8447054849882516

In [36]:
test_results.r2

0.9460048386258194

In [38]:
assembler2 = VectorAssembler(inputCols=['passengers', 'length','cabins'], outputCol = 'features')

In [39]:
final_data2 = assembler2.transform(df_index)

In [44]:
final_data2.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- Cruise_line_index: double (nullable = true)
 |-- features: vector (nullable = true)



In [40]:
output_data2 = final_data2.select('features', 'crew')

In [45]:
output_data2.show()

+------------------+----+
|          features|crew|
+------------------+----+
|  [6.94,5.94,3.55]|3.55|
|  [6.94,5.94,3.55]|3.55|
| [14.86,7.22,7.43]| 6.7|
|[29.74,9.53,14.88]|19.1|
|[26.42,8.92,13.21]|10.0|
| [20.52,8.55,10.2]| 9.2|
| [20.52,8.55,10.2]| 9.2|
|[20.56,8.55,10.22]| 9.2|
| [20.52,8.55,10.2]| 9.2|
| [37.0,9.51,14.87]|11.5|
|[29.74,9.51,14.87]|11.6|
| [14.52,7.27,7.26]| 6.6|
| [20.52,8.55,10.2]| 9.2|
| [20.52,8.55,10.2]| 9.2|
|[21.24,9.63,10.62]| 9.3|
|[29.74,9.51,14.87]|11.6|
|[21.24,9.63,10.62]|10.3|
| [20.52,8.55,10.2]| 9.2|
|[21.24,9.63,11.62]| 9.3|
| [20.52,8.55,10.2]| 9.2|
+------------------+----+
only showing top 20 rows



In [41]:
train, test = output_data2.randomSplit([0.7,0.3])

In [46]:
lr2 = LinearRegression(labelCol='crew')

In [47]:
lr_model2 = lr2.fit(train)

In [48]:
lr_model2.summary.pValues

[0.0004947455080668295, 0.0003123149839305839, 0.0, 0.11921898519364116]

In [49]:
test_results2 = lr_model2.evaluate(test)

In [50]:
test_results2.rootMeanSquaredError

1.115583028088497

In [51]:
test_results2.r2

0.8949334975334914

In [52]:
from pyspark.sql.functions import corr

In [55]:
df.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [56]:
df.select(corr('crew', 'passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [57]:
df.select(corr('crew', 'Age')).show()

+-------------------+
|    corr(crew, Age)|
+-------------------+
|-0.5306565039638852|
+-------------------+



In [58]:
df.select(corr('crew', 'Tonnage')).show()

+-------------------+
|corr(crew, Tonnage)|
+-------------------+
|  0.927568811544939|
+-------------------+



In [59]:
df.select(corr('cabins', 'passengers')).show()

+------------------------+
|corr(cabins, passengers)|
+------------------------+
|      0.9763413679845945|
+------------------------+

